In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define the function to simulate one day with a specific delta
def simulate_day(delta):
    BES_capacity = 150  # Backup Energy Source (BES) capacity in kWh
    BES_current = BES_capacity  # Start with fully charged BES
    L = 3.0  # Initial lower temperature limit
    U = 7.0  # Upper temperature limit

    res_used_total = 0
    bes_used_total = 0
    grid_used_total = 0
    
    # Iterate through each time step in the day's data
    for _, row in merged_data.iterrows():
        res_available = row['GES(inKWh)']  # Renewable energy available
        fridge_power_demand = row['Power_final']  # Power demand

        # Check if RES is sufficient
        if res_available >= fridge_power_demand:
            res_used_total += fridge_power_demand
        else:
            # Partial RES usage if RES is insufficient
            res_used_total += res_available
            remaining_demand = fridge_power_demand - res_available
            
            # Adjust L towards U if RES is zero and L < U
            if res_available == 0 and L < U:
                L = min(L + delta, U)
            
            # Use BES if it can cover the remaining demand
            if BES_current >= remaining_demand:
                bes_used_total += remaining_demand
                BES_current -= remaining_demand
            else:
                # Use all BES, then Grid for remaining demand
                bes_used_total += BES_current
                grid_used_total += (remaining_demand - BES_current)
                BES_current = 0

    # Calculate a score as a function of energy sources used (lower is better)
    score = - (res_used_total + 0.5 * bes_used_total - 1.5 * grid_used_total)
    
    return res_used_total, bes_used_total, grid_used_total, score

# Example usage of simulate_day
deltas = np.arange(0.05, 0.55, 0.05)  # Example delta values
data = []

# Define BES_capacity before the loop
BES_capacity = 150

for delta in deltas:
    res_used, bes_used, grid_used, score = simulate_day(delta)
    daily_res = np.sum(merged_data['GES(inKWh)'])  # Total RES for the day
    avg_demand = np.mean(merged_data['Power_final'])  # Average power demand for the day
    
    # Collect features and target
    data.append([daily_res, avg_demand, BES_capacity, delta, res_used, bes_used, grid_used, score])

# Convert to DataFrame
columns = ['Daily_RES', 'Avg_Demand', 'BES_Capacity', 'Delta', 'RES_used', 'BES_used', 'Grid_used', 'Score']
df = pd.DataFrame(data, columns=columns)

# Step 2: Define features and target variable
X = df[['Daily_RES', 'Avg_Demand', 'BES_Capacity', 'RES_used', 'BES_used', 'Grid_used']]
y = df['Delta']

# Step 3: Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Prepare today's data
today_data = pd.DataFrame([[daily_res, avg_demand, BES_capacity, res_used, bes_used, grid_used]], 
                          columns=['Daily_RES', 'Avg_Demand', 'BES_Capacity', 'RES_used', 'BES_used', 'Grid_used'])
predicted_delta = model.predict(today_data)
print("Predicted optimal delta:", predicted_delta[0])


Mean Squared Error: 0.030626
Predicted optimal delta: 0.2739999999999999
